In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from datasets import Dataset
from huggingface_hub import login
login()

In [3]:
import pandas as pd

splits = {'train': 'training set.csv', 'test': 'testing set.csv'}
df1 = pd.read_csv("hf://datasets/thefrankhsu/hate_speech_twitter/" + splits["train"])

In [4]:
df1['categories'] = df1['categories'].fillna("Normal")

In [5]:
df1['categories'] = df1['categories'].replace({'Normal':0, 'Race':1, 'Sexual Orientation':2, 'Gender':3, 'Physical Appearance':4, 'Religion':5, 'Disability':6, 'Class':7, 'Ethnicity':8, 'Behavior':9})

C:\Users\Mrudula\AppData\Local\Temp\ipykernel_23044\1376013957.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['categories'] = df1['categories'].replace({'Normal':0, 'Race':1, 'Sexual Orientation':2, 'Gender':3, 'Physical Appearance':4, 'Religion':5, 'Disability':6, 'Class':7, 'Ethnicity':8, 'Behavior':9})


In [6]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(preserve_case=True)

In [7]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
remove_words = list(set(stop_words)-set(['no','never']))
def remove_stopwords(text):

    clean_text = [word for word in text if not word in remove_words]
    return clean_text   

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download necessary NLTK data files
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional: for additional language support
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mrudula\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Mrudula\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mrudula\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mrudula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mrudula\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from nltk.tag import pos_tag
def get_wordnet_pos(treebank_tag):
    """Convert treebank POS tags to WordNet POS tags."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if POS is unknown

def lemmatize_sentence(sentence):
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence)
    # Get POS tags for each word
    pos_tags = pos_tag(words)
    # Lemmatize each word with the correct POS tag
    lemmatized_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return ' '.join(lemmatized_sentence)

In [10]:
import re
replacement_map = {
    r'\bfuck(ing|in|ry)?\b': 'fuck',
    r'\bn(igg|igga|iggah|ighas)\b': 'nigger',
    r'\bfag(got)?\b': 'faggot'
}

# Function to apply replacements
def replace_words(text):
    for pattern, replacement in replacement_map.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

In [11]:
def simplify(text):
    '''Function to handle the diacritics in the text'''
    import unicodedata
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text)

df1['Tweet'] = df1['tweet'].astype('str').apply(simplify)
df1['Tweet'] = df1['Tweet'].replace(r'@\w+','',regex=True)
df1['Tokenized_tweet'] = df1['Tweet'].apply(tokenizer.tokenize)
df1['Clean_tweets'] = df1['Tokenized_tweet'].apply(remove_stopwords)
df1['Clean_tweets'] = df1['Clean_tweets'].str.join(' ')
df1['Lemmatized_Tweets'] = df1['Clean_tweets'].apply(lemmatize_sentence)
df1['Lemmatized_Tweets'] = df1['Lemmatized_Tweets'].apply(replace_words)


In [12]:
X_labeled = df1['Lemmatized_Tweets']
y_labeled = df1['categories']

In [13]:
df2 = pd.read_csv('instagram_comments.csv')

In [14]:
def convert_to_lower(text):
    return text.lower()

In [15]:
replacement_map = {
    "fuck": ["fuckhes", "fu**s", "eff", "f", "fook", "falk", "phuck"],
    "fucked": ["fcked"],
    "as fuck": ["af"],
    "shit": ["sh$t", "sh!t", "s**t"],
    "what the fuck": ["wtf", "wtfff", "dafuq"],
    "bitch": ["b*tch", "bich", "b*tches"],
    "get the fuck off": ["gtfo"],
    "fuckers": ["f*ckers"],
    "what the actual fuck": ["wtaf"],
    "fuck this": ["fuckthis"],
    "fuck justin": ["fuckjustin"],
    "fuck him": ["fuckhim"],
    "shame saudi arabia": ["shamesaudiarabia"],
    "the fuck": ["tf"],
    "sucking": ["su*king"],
    "dick": ["di*k"],
    "fucking": ["fuckin", "effin", "effing", "fn"],
    "fuck your": ["fuckyour"],
    "sucks": ["s*cks"],
    "trash": ["tra$hh"],
    "pussy": ["pu$$y"],
    "third": ["3rd"],
    "you have": ["you've"],
    "did not": ["didn't"],
    "do not": ["don't"],
    "were not": ["weren't"],
    "where is": ["where's"],
    "fuck democracy": ["fuckdemocracy"],
    "gaza": ["g@.za"],
    "nigger": ["nigga","niggas"],
    "mother fucker": ["mf"],
    "please": ["plz"],
    "you": ["u"],
    "you are": ["you're"],
    "end israeli apartheid": ["endisraeliapartheid"],
    "palestine": ["palest1ne"],
    "i don't know": ["idk"],
    "to be honest": ["tbh"],
    "your": ["ur"],
    "you have": ["you've"],
    "are not": ["aren't"],
    "nepotism sucks": ["nepotismsucks"],
    "what the hell": ["wth"],
    "be kind": ["bekind"],
    "stop trolling": ["stoptrolling"],
    "i have": ["i've"],
    "was not": ["wasn't"],
    "shut the fuck up": ["stfu"],
    "i know": ["ik"],
    "you all": ["y'all"],
    "ass hole": ["a.hole", "a-hole"],
    "rappist": ["rappst"],
    "gay": ["g@ys"],
    "pedophile": ["pedo"],
    "hate": ["h8te"],
    "zionists": ["zi0nists"],
    "holocaust": ["hol0caust"]
}
def replace_words(text, replacement_map):
    # Sort variations by length (longer first) to handle overlapping replacements
    sorted_map = {}
    for key, variations in replacement_map.items():
        for variation in sorted(variations, key=len, reverse=True):
            sorted_map[variation] = key

    # Create a single regex pattern for all variations
    pattern = re.compile(r'\b(?:' + '|'.join(re.escape(var) for var in sorted_map.keys()) + r')\b', flags=re.IGNORECASE)

    # Replace matches using the sorted map
    def replace_match(match):
        matched_text = match.group(0).lower()  # Match case-insensitively
        return sorted_map.get(matched_text, matched_text)

    return pattern.sub(replace_match, text)

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
remove_words = list(set(stop_words)-set(['never','no','not']))
def remove_stopwords(text):

    clean_text = [word for word in text if not word in remove_words]
    return clean_text  

In [18]:
df2['Text'] = df2['Comment'].apply(lambda x: convert_to_lower(x))
df2['Text'] = df2['Text'].apply(lambda x: x.strip().replace('\n', ' ').replace('\r', ' '))
df2['Text'] = df2['Text'].apply(lambda x: re.sub(r'&#039;', '\'', x))
df2['Text'] = df2['Text'].apply(lambda x: re.sub(r'&[^;]*;', '', x))
df2['Text'] = df2['Text'] \
    .apply(lambda x: re.sub(r'[^a-zA-Z\'\.\,]', ' ', x)) \
    .apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x)) \
    .apply(lambda x: re.sub(r'\.{2,}', '.', x))
df2['Text'] = df2['Text'].apply(lambda x: re.sub(r'http\S+', '', x))
df2['Cleaner_Text'] = df2['Text'].str.lower().apply(tokenizer.tokenize)
df2['Clean_tweets'] = df2['Cleaner_Text'].apply(remove_stopwords)
df2['Clean_tweets'] = df2['Clean_tweets'].str.join(' ')
df2['Lemmatized_Tweets'] = df2['Clean_tweets'].apply(lemmatize_sentence)

In [19]:
df2['Categories'] = -1

In [20]:
X_unlabeled = df2['Lemmatized_Tweets']
y_unlabeled = df2['Categories']

In [21]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.semi_supervised import LabelSpreading, SelfTrainingClassifier
from sklearn.metrics import classification_report

In [22]:
sdg_params = dict(alpha=1e-5, penalty="l2", loss="log_loss")
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

In [31]:
#Supervised
pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SGDClassifier(**sdg_params)),
    ]
)

#Unsupervised
st_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SelfTrainingClassifier(SGDClassifier(**sdg_params))),
    ]
)

ls_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        # LabelSpreading does not support dense matrices
       # ("toarray", FunctionTransformer(lambda x: x.toarray())),
        ("clf", LabelSpreading(kernel="knn", alpha=0.9)),
    ]
)

In [32]:

def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:", sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(
        "Micro-averaged F1 score on test set: %0.3f"
        % f1_score(y_test, y_pred, average="weighted")
    )
    print("-" * 10)
    print()

In [33]:
X_combined = pd.concat([pd.Series(X_labeled), pd.Series(X_unlabeled)], ignore_index=True)
y_combined = np.concatenate([y_labeled, y_unlabeled])


In [34]:
if __name__ == "__main__":
    X, y = X_combined, y_combined
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    print("Supervised SGDClassifier on 100% of the data:")
    eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)
    
    print(classification_report(y_test, pipeline.predict(X_test)))

    # select a mask of 20% of the train dataset
    y_mask = np.random.rand(len(y_train)) < 0.2

    # X_20 and y_20 are the subset of the train dataset indicated by the mask
    X_20, y_20 = map(list, zip(*((x, y)
                     for x, y, m in zip(X_train, y_train, y_mask) if m)))
    print("Supervised SGDClassifier on 20% of the training data:")
    eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)
    
    print(classification_report(y_test, pipeline.predict(X_test)))

    # set the non-masked subset to be unlabeled
    y_train[~y_mask] = -1
    print("SelfTrainingClassifier on 20% of the training data (rest "
          "is unlabeled):")
    eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)
    
    print(classification_report(y_test, st_pipeline.predict(X_test)))

Supervised SGDClassifier on 100% of the data:
Number of training samples: 8518
Unlabeled samples in training set: 4251
Micro-averaged F1 score on test set: 0.883
----------

              precision    recall  f1-score   support

          -1       0.91      0.98      0.94      1428
           0       0.90      0.90      0.90      1034
           1       0.73      0.59      0.65       147
           2       0.96      0.87      0.91       101
           3       0.61      0.52      0.56        67
           4       0.25      0.08      0.12        13
           5       0.00      0.00      0.00        10
           6       0.50      0.29      0.36         7
           7       1.00      0.27      0.42        15
           8       0.00      0.00      0.00         9
           9       0.00      0.00      0.00         9

    accuracy                           0.89      2840
   macro avg       0.53      0.41      0.44      2840
weighted avg       0.88      0.89      0.88      2840

Supervised SG

C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

Micro-averaged F1 score on test set: 0.274
----------

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1428
           0       0.39      0.99      0.56      1034
           1       0.74      0.46      0.57       147
           2       0.96      0.73      0.83       101
           3       0.57      0.36      0.44        67
           4       0.00      0.00      0.00        13
           5       0.00      0.00      0.00        10
           6       0.20      0.14      0.17         7
           7       0.00      0.00      0.00        15
           8       1.00      0.11      0.20         9
           9       0.00      0.00      0.00         9

    accuracy                           0.42      2840
   macro avg       0.35      0.25      0.25      2840
weighted avg       0.23      0.42      0.27      2840



C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [39]:
#sdg_params = dict(alpha=1e-5, penalty="l2", loss="log_loss")
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)
sdg_params = {"loss": "log_loss", "max_iter": 1000, "tol": 1e-3, "random_state": 42}
rf_params = {"n_estimators": 100, "max_depth": 20, "random_state": 42}
svc_params = {"kernel": "linear", "probability": True, "C": 1.0}


In [36]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SGDClassifier(**sdg_params)),
    ]
)

#Unsupervised
st_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SelfTrainingClassifier(RandomForestClassifier(**rf_params))),
    ]
)

ls_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        # LabelSpreading does not support dense matrices
       # ("toarray", FunctionTransformer(lambda x: x.toarray())),
        ("clf", LabelSpreading(kernel="knn", alpha=0.8)),
    ]
)

In [41]:
if __name__ == "__main__":
    X, y = X_combined, y_combined
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    print("Supervised SGDClassifier on 100% of the data:")
    eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)
    
    print(classification_report(y_test, pipeline.predict(X_test)))

    # select a mask of 20% of the train dataset
    y_mask = np.random.rand(len(y_train)) < 0.2

    # X_20 and y_20 are the subset of the train dataset indicated by the mask
    X_20, y_20 = map(list, zip(*((x, y)
                     for x, y, m in zip(X_train, y_train, y_mask) if m)))
    print("Supervised SGDClassifier on 20% of the training data:")
    eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)
    
    print(classification_report(y_test, pipeline.predict(X_test)))

    # set the non-masked subset to be unlabeled
    y_train[~y_mask] = -1
    print("SelfTrainingClassifier on 20% of the training data (rest "
          "is unlabeled):")
    eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)
    
    print(classification_report(y_test, st_pipeline.predict(X_test)))

Supervised SGDClassifier on 100% of the data:
Number of training samples: 8518
Unlabeled samples in training set: 4258
Micro-averaged F1 score on test set: 0.869
----------



C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

              precision    recall  f1-score   support

          -1       0.88      0.98      0.93      1421
           0       0.90      0.89      0.89      1033
           1       0.74      0.47      0.58       123
           2       0.97      0.85      0.90       110
           3       0.72      0.54      0.61        80
           4       0.00      0.00      0.00        17
           5       0.00      0.00      0.00        10
           6       0.75      0.27      0.40        11
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00        14

    accuracy                           0.88      2840
   macro avg       0.45      0.36      0.39      2840
weighted avg       0.86      0.88      0.87      2840

Supervised SGDClassifier on 20% of the training data:
Number of training samples: 1709
Unlabeled samples in training set: 865
Micro-averaged F1 score on test set: 0.804
----------

      

C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mrudula\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz